# Fuck pandas!

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [5]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [6]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

In [7]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [8]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population']).fillna(0)
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population']).fillna(0)

In [9]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [10]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [11]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [12]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [13]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [14]:
X_train.loc[:,'wpt_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'wpt_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [15]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [16]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [17]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [18]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [19]:
drop_cats = ['source_type', 'source_class',
              'water_quality', 'quality_group',
             'quantity_group', 'waterpoint_type', 'waterpoint_type_group',
              'payment_type', 'extraction_type_group',
             'extraction_type_class', 'management_group'
              ]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [20]:
X_train.iloc[0,:]

amount_tsh                                              6000
funder                                                 Roman
gps_height                                              1390
installer                                              Roman
longitude                                            34.9381
latitude                                            -9.85632
wpt_name                                                   1
basin                                             Lake Nyasa
subvillage                                          Mnyusi B
region                                                Iringa
region_code                                               11
district_code                                              5
lga                                                   Ludewa
ward                                                Mundindi
population                                               109
public_meeting                                             1
scheme_management       

In [21]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region',
                'lga', 'ward', 'quality',
                'scheme_management', 'scheme_name',
                'extraction_type',
                'management', 'payment',
                'quantity',
                'source', 'wpt_type',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []

In [22]:
for cat in text_features:
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        X_train.loc[:,cat].fillna(0)
        X_test.loc[:,cat].fillna(0)
    labels = X_test.loc[:,cat].unique()
    labelscnt = np.zeros((3, labels.size))
    def label_to_i(lbl):
        if  pd.isnull(lbl) or lbl == 'unknown' or lbl == 'other' or lbl == 'others' or lbl == 0:
            return 0
        i = labels.searchsorted(lbl)
        if i < labels.size and labels[i] == lbl:
            return 1 + i
        return 0
    X_train.loc[:,cat] = X_train.loc[:,cat].map(label_to_i)
    X_test.loc[:,cat] = X_test.loc[:,cat].map(label_to_i)
    
#    tr = pd.DataFrame(v.transform(X_train.loc[:,cat]).toarray()).add_prefix(cat + '_')
#    te = pd.DataFrame(v.transform(X_test.loc[:,cat]).toarray()).add_prefix(cat + '_')
#    text_feats_train.append(tr)
#    text_feats_test.append(te)



#X_train = pd.concat([X_train.drop(text_features, axis=1, errors='ignore')] + text_feats_train,
#                    axis=1)
#X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore')] + text_feats_test,
#                    axis=1)


In [23]:
X_train.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,...,payment,quantity,source,date_recorded_year,date_recorded_month,date_recorded_day,age,amount_per_man,quality,wpt_type
0,6000.0,0,1390,0,34.938093,-9.856322,1,0,0,0,...,0,4,0,2011,3,14,12,55.045872,0,0
1,0.0,0,1399,0,34.698766,-2.147466,1,0,0,0,...,1,0,1,2013,3,6,3,0.000000,0,0
2,25.0,0,686,0,37.460664,-3.821329,1,0,0,0,...,0,4,0,2013,2,25,4,0.100000,0,0
3,0.0,0,263,0,38.486161,-11.155298,1,0,0,0,...,1,0,0,2013,1,28,27,0.000000,0,0
4,0.0,0,0,0,31.130847,-1.825359,1,0,0,0,...,1,0,1,2011,7,13,2011,0.000000,0,0


In [24]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[]

In [25]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])
xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(X_train, ytr, proportions=(.7, .1, .2))
)
y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)
display(
    np.where(ytr != 0)[0][:40],
    np.where(ycv != 0)[0][:40],
    np.where(yva != 0)[0][:40],)
ytr = y_bin.transform(ytr)
ycv = y_bin.transform(ycv)
yva = y_bin.transform(yva)

array([22581, 22582, 22583, 22584, 22585, 22586, 22587, 22588, 22589,
       22590, 22591, 22592, 22593, 22594, 22595, 22596, 22597, 22598,
       22599, 22600, 22601, 22602, 22603, 22604, 22605, 22606, 22607,
       22608, 22609, 22610, 22611, 22612, 22613, 22614, 22615, 22616,
       22617, 22618, 22619, 22620])

array([3225, 3226, 3227, 3228, 3229, 3230, 3231, 3232, 3233, 3234, 3235,
       3236, 3237, 3238, 3239, 3240, 3241, 3242, 3243, 3244, 3245, 3246,
       3247, 3248, 3249, 3250, 3251, 3252, 3253, 3254, 3255, 3256, 3257,
       3258, 3259, 3260, 3261, 3262, 3263, 3264])

array([6453, 6454, 6455, 6456, 6457, 6458, 6459, 6460, 6461, 6462, 6463,
       6464, 6465, 6466, 6467, 6468, 6469, 6470, 6471, 6472, 6473, 6474,
       6475, 6476, 6477, 6478, 6479, 6480, 6481, 6482, 6483, 6484, 6485,
       6486, 6487, 6488, 6489, 6490, 6491, 6492])

In [26]:
def y_to_df(y, ids):
    y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df

In [27]:
X_train.columns.tolist()

['amount_tsh',
 'funder',
 'gps_height',
 'installer',
 'longitude',
 'latitude',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'population',
 'public_meeting',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'management',
 'payment',
 'quantity',
 'source',
 'date_recorded_year',
 'date_recorded_month',
 'date_recorded_day',
 'age',
 'amount_per_man',
 'quality',
 'wpt_type']

In [28]:
X_train.to_csv('X_train_prepr.csv')
X_test.to_csv('X_test_prepr.csv')
pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr.csv')
pd.Series(y_enc1.classes_).to_csv('y_classes.csv')

In [29]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.utils

Using TensorFlow backend.


In [30]:
m = Sequential([
    Dense(2*xtr.shape[1], input_shape=(xtr.shape[1],), activation='softmax'),
    Dropout(.4),
    Dense(2*xtr.shape[1], activation='softmax'),
    Dense(2*xtr.shape[1], activation='softmax'),
    Dense(3, activation='softmax'),
])

In [31]:
m.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [32]:
m.fit(xtr.values, ytr,
      batch_size=512,
      epochs=500)

Epoch 1/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 2/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 3/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 4/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 5/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 6/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 7/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 8/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 9/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 10/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 11/500
41578/41578 [==============================] - 0s - loss

41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 89/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 90/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 91/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 92/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 93/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 94/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 95/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 96/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 97/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 98/500
41578/41578 [==============================] - 0s - loss: na

41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 175/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 176/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 177/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 178/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 179/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 180/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 181/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 182/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 183/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 184/500
41578/41578 [==============================] - 0s 

41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 261/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 262/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 263/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 264/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 265/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 266/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 267/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 268/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 269/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 270/500
41578/41578 [==============================] - 0s 

41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 347/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 348/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 349/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 350/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 351/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 352/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 353/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 354/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 355/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 356/500
41578/41578 [==============================] - 0s 

41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 433/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 434/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 435/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 436/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 437/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 438/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 439/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 440/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 441/500
41578/41578 [==============================] - 0s - loss: nan - acc: 0.5431     
Epoch 442/500
41578/41578 [==============================] - 0s 

In [33]:
import sklearn.metrics

In [36]:
m.predict(xva.values)

array([[ nan,  nan,  nan],
       [ nan,  nan,  nan],
       [ nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan],
       [ nan,  nan,  nan],
       [ nan,  nan,  nan]], dtype=float32)